<a href="https://colab.research.google.com/github/Aggarwal-Bhavya/Food-Delivery-Cost-Analysis/blob/main/Food_Delivery_Cost_%26_Profability_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'food-delivery-cost-analysis:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F5526922%2F9149896%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240811%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240811T100708Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D51187f49f64bd67411dc53894046119cf31a3cf87ed5856acb8947bdedcf49f3947962f05b359fcf08e364bf39544894be1260978667a1f3e459c28f543ad71b308c0c2beac2afdd485bd88acd6461343501bec3124cfa7e84e2cb423718d145113a6f142e83b001ea1d05afa04b27ab5b72bbd2838564581d512f35fcc8b9fc5a9122e12e2086b389df4495888167e68b8b070757c5292bd43d3b02d575a854ef64641f8b1c43ee6cafd1c242e35c225af9328fb037fee308509577f52565e49902771dfc5ffe4d22c9bfb42bc9fb348d60c22a38b5fc2871519e8591c17c92aefec0e5d0d0c5e82d58120c4806079bb19e23009a25def85fa19774160f3ec2'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 28628 bytes downloaded
Downloaded and uncompressed: food-delivery-cost-analysis
Data source import complete.


# Executive Summary

In this report, we analyze the cost, revenue, and profitability of our business operations to identify key areas for improvement. Based on our findings, we propose strategies to reduce costs and boost revenue, and simulate their impact on overall profitability. The simulation results suggest significant improvements in profit margins, indicating the effectiveness of the proposed strategies.

# Background and Context

A food delivery service has been experiencing fluctuating profits, prompting a detailed analysis to identify cost drivers and revenue opportunities.

# Objectives

* To analyze cost, revenue, and profitability.
* To propose strategies for cost reduction and revenue enhancement.
* To simulate the impact of these strategies on overall profitability.

# Data Overview

**DataSet Description**:

The dataset includes order-level data with variables such as order value, discounts, payment processing fees, delivery fees, commission fees, and profit.

**Key Variables**:
* **Order ID**: unique indentifier for each order
* **Customer ID**: unique identifier for each customer
* **Restaurant ID**: unique identifier for each restaurant
* **Order Date and Time**: timestamp indicating when order was placed
* **Delivery Date and Time**: timestamp indicating when order was delivered
* **Order Value**: total value of the order
* **Delivery Fee**: fee charged for delivering the order
* **Payment Method**: method used by the customer to pay for the order
* **Discounts and Offers**: type of discount or promotional offers applied to the offer
* **Commission Fee**: revenue earned from commissions charged to the restaurant
* **Payment Processing Fee**: fee charged for processing the customer's payment
* **Refunds and Chargebacks**: amount refunded to the customer or chargebacks incurred

In [2]:
# importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil.relativedelta import relativedelta
pd.options.display.float_format = '{:.2f}'.format

# Reading the data set

In [3]:
food_delivery_cost = pd.read_csv('/kaggle/input/food-delivery-cost-analysis/food delivery costs.csv')
food_delivery_cost.head()

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks
0,1,C8270,R2924,2024-02-01 01:11:52,2024-02-01 02:39:52,1914,0,Credit Card,5% on App,150,47,0
1,2,C1860,R2054,2024-02-02 22:11:04,2024-02-02 22:46:04,986,40,Digital Wallet,10%,198,23,0
2,3,C6390,R2870,2024-01-31 05:54:35,2024-01-31 06:52:35,937,30,Cash on Delivery,15% New User,195,45,0
3,4,C6191,R2642,2024-01-16 22:52:49,2024-01-16 23:38:49,1463,50,Cash on Delivery,NaN,146,27,0
4,5,C6734,R2799,2024-01-29 01:19:30,2024-01-29 02:48:30,1992,30,Cash on Delivery,50 off Promo,130,50,0


In [4]:
food_delivery_cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Order ID                1000 non-null   int64 
 1   Customer ID             1000 non-null   object
 2   Restaurant ID           1000 non-null   object
 3   Order Date and Time     1000 non-null   object
 4   Delivery Date and Time  1000 non-null   object
 5   Order Value             1000 non-null   int64 
 6   Delivery Fee            1000 non-null   int64 
 7   Payment Method          1000 non-null   object
 8   Discounts and Offers    815 non-null    object
 9   Commission Fee          1000 non-null   int64 
 10  Payment Processing Fee  1000 non-null   int64 
 11  Refunds/Chargebacks     1000 non-null   int64 
dtypes: int64(6), object(6)
memory usage: 93.9+ KB


In [5]:
food_delivery_cost.describe()

,Order ID,Order Value,Delivery Fee,Commission Fee,Payment Processing Fee,Refunds/Chargebacks
count,1000.00,1000.00,1000.00,1000.00,1000.00,1000.00
mean,500.50,1053.97,28.62,126.99,29.83,28.30
std,288.82,530.98,16.96,43.06,11.63,49.61
min,1.00,104.00,0.00,50.00,10.00,0.00
25%,250.75,597.75,20.00,90.00,20.00,0.00
50%,500.50,1038.50,30.00,127.00,30.00,0.00
75%,750.25,1494.00,40.00,164.00,40.00,50.00
max,1000.00,1995.00,50.00,200.00,50.00,150.00


In [6]:
food_delivery_cost['Payment Method'].value_counts()

,count
Payment Method,
Cash on Delivery,357
Credit Card,337
Digital Wallet,306


# Data Cleaning

Standardizing the **date time formats** in the dataset

In [7]:
# Converting the Order Date and Time to standardized datetime format
food_delivery_cost['Order Date and Time'] = pd.to_datetime(food_delivery_cost['Order Date and Time'])
food_delivery_cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Order ID                1000 non-null   int64         
 1   Customer ID             1000 non-null   object        
 2   Restaurant ID           1000 non-null   object        
 3   Order Date and Time     1000 non-null   datetime64[ns]
 4   Delivery Date and Time  1000 non-null   object        
 5   Order Value             1000 non-null   int64         
 6   Delivery Fee            1000 non-null   int64         
 7   Payment Method          1000 non-null   object        
 8   Discounts and Offers    815 non-null    object        
 9   Commission Fee          1000 non-null   int64         
 10  Payment Processing Fee  1000 non-null   int64         
 11  Refunds/Chargebacks     1000 non-null   int64         
dtypes: datetime64[ns](1), int64(6), object(5)
memory 

In [8]:
# Converting the Delivery Date and Time to standardized datetime format
food_delivery_cost['Delivery Date and Time'] = pd.to_datetime(food_delivery_cost['Delivery Date and Time'])
food_delivery_cost.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Order ID                1000 non-null   int64         
 1   Customer ID             1000 non-null   object        
 2   Restaurant ID           1000 non-null   object        
 3   Order Date and Time     1000 non-null   datetime64[ns]
 4   Delivery Date and Time  1000 non-null   datetime64[ns]
 5   Order Value             1000 non-null   int64         
 6   Delivery Fee            1000 non-null   int64         
 7   Payment Method          1000 non-null   object        
 8   Discounts and Offers    815 non-null    object        
 9   Commission Fee          1000 non-null   int64         
 10  Payment Processing Fee  1000 non-null   int64         
 11  Refunds/Chargebacks     1000 non-null   int64         
dtypes: datetime64[ns](2), int64(6), object(4)
memory 

In [9]:
food_delivery_cost.head()

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks
0,1,C8270,R2924,2024-02-01 01:11:52,2024-02-01 02:39:52,1914,0,Credit Card,5% on App,150,47,0
1,2,C1860,R2054,2024-02-02 22:11:04,2024-02-02 22:46:04,986,40,Digital Wallet,10%,198,23,0
2,3,C6390,R2870,2024-01-31 05:54:35,2024-01-31 06:52:35,937,30,Cash on Delivery,15% New User,195,45,0
3,4,C6191,R2642,2024-01-16 22:52:49,2024-01-16 23:38:49,1463,50,Cash on Delivery,NaN,146,27,0
4,5,C6734,R2799,2024-01-29 01:19:30,2024-01-29 02:48:30,1992,30,Cash on Delivery,50 off Promo,130,50,0


Adding **Order Fulfillment Time** parameter in minutes which processes the time taken from the time of Order Placement to Successful Delivery of an order.

In [10]:
# Adding a new column to calculate the time taken to fulfill an order aka from it being place
# to it being delivered (in minutes)
def calculate_duration (start, end):
    duration = relativedelta(end, start)
    return duration.hours*60 + duration.minutes

food_delivery_cost['Order Fulfillment Time'] = food_delivery_cost.apply(lambda row: calculate_duration(row['Order Date and Time'], row['Delivery Date and Time']), axis = 1)
food_delivery_cost.head()

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks,Order Fulfillment Time
0,1,C8270,R2924,2024-02-01 01:11:52,2024-02-01 02:39:52,1914,0,Credit Card,5% on App,150,47,0,88
1,2,C1860,R2054,2024-02-02 22:11:04,2024-02-02 22:46:04,986,40,Digital Wallet,10%,198,23,0,35
2,3,C6390,R2870,2024-01-31 05:54:35,2024-01-31 06:52:35,937,30,Cash on Delivery,15% New User,195,45,0,58
3,4,C6191,R2642,2024-01-16 22:52:49,2024-01-16 23:38:49,1463,50,Cash on Delivery,NaN,146,27,0,46
4,5,C6734,R2799,2024-01-29 01:19:30,2024-01-29 02:48:30,1992,30,Cash on Delivery,50 off Promo,130,50,0,89


In [11]:
# checking if all rows have received some values
food_delivery_cost['Order Fulfillment Time'].isnull().sum()

0

Processing the **Order Date and Time** for use in the EDA to fetch different aspects of the order making process.
* Categorizing order_time based on hour
* Extracting order_date
* Extracting order_day

In [12]:
# Extracting relevant information from Order Date and Time
# 1. Categorizing order_time based on hour
def categorize_order_time (hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 16:
        return 'Afternoon'
    elif 16 <= hour < 20:
        return 'Evening'
    elif 20 <= hour < 24:
        return 'Night'
    else:
        return 'Midnight'

food_delivery_cost['order_time'] = food_delivery_cost['Order Date and Time'].dt.hour.apply(categorize_order_time)
food_delivery_cost.head()

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks,Order Fulfillment Time,order_time
0,1,C8270,R2924,2024-02-01 01:11:52,2024-02-01 02:39:52,1914,0,Credit Card,5% on App,150,47,0,88,Midnight
1,2,C1860,R2054,2024-02-02 22:11:04,2024-02-02 22:46:04,986,40,Digital Wallet,10%,198,23,0,35,Night
2,3,C6390,R2870,2024-01-31 05:54:35,2024-01-31 06:52:35,937,30,Cash on Delivery,15% New User,195,45,0,58,Morning
3,4,C6191,R2642,2024-01-16 22:52:49,2024-01-16 23:38:49,1463,50,Cash on Delivery,NaN,146,27,0,46,Night
4,5,C6734,R2799,2024-01-29 01:19:30,2024-01-29 02:48:30,1992,30,Cash on Delivery,50 off Promo,130,50,0,89,Midnight


In [13]:
# 2. Extracting order_date
food_delivery_cost['order_date'] = food_delivery_cost['Order Date and Time'].dt.date
food_delivery_cost.head()

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks,Order Fulfillment Time,order_time,order_date
0,1,C8270,R2924,2024-02-01 01:11:52,2024-02-01 02:39:52,1914,0,Credit Card,5% on App,150,47,0,88,Midnight,2024-02-01
1,2,C1860,R2054,2024-02-02 22:11:04,2024-02-02 22:46:04,986,40,Digital Wallet,10%,198,23,0,35,Night,2024-02-02
2,3,C6390,R2870,2024-01-31 05:54:35,2024-01-31 06:52:35,937,30,Cash on Delivery,15% New User,195,45,0,58,Morning,2024-01-31
3,4,C6191,R2642,2024-01-16 22:52:49,2024-01-16 23:38:49,1463,50,Cash on Delivery,NaN,146,27,0,46,Night,2024-01-16
4,5,C6734,R2799,2024-01-29 01:19:30,2024-01-29 02:48:30,1992,30,Cash on Delivery,50 off Promo,130,50,0,89,Midnight,2024-01-29


In [14]:
# 3. Extracting order_day
food_delivery_cost['order_day'] = food_delivery_cost['Order Date and Time'].dt.day_name()
food_delivery_cost.head()

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks,Order Fulfillment Time,order_time,order_date,order_day
0,1,C8270,R2924,2024-02-01 01:11:52,2024-02-01 02:39:52,1914,0,Credit Card,5% on App,150,47,0,88,Midnight,2024-02-01,Thursday
1,2,C1860,R2054,2024-02-02 22:11:04,2024-02-02 22:46:04,986,40,Digital Wallet,10%,198,23,0,35,Night,2024-02-02,Friday
2,3,C6390,R2870,2024-01-31 05:54:35,2024-01-31 06:52:35,937,30,Cash on Delivery,15% New User,195,45,0,58,Morning,2024-01-31,Wednesday
3,4,C6191,R2642,2024-01-16 22:52:49,2024-01-16 23:38:49,1463,50,Cash on Delivery,NaN,146,27,0,46,Night,2024-01-16,Tuesday
4,5,C6734,R2799,2024-01-29 01:19:30,2024-01-29 02:48:30,1992,30,Cash on Delivery,50 off Promo,130,50,0,89,Midnight,2024-01-29,Monday
